### Theano example implementation for data classification

This example should illustrate the functionality of Theano on the TensorFlow examples.

In [ ]:
# Importing all the things
%matplotlib notebook

import theano
import numpy as np
import theano.tensor as T
import numpy.random as rnd
import matplotlib.pyplot as plt

from numpy import linalg as la

In [ ]:
uniform = lambda a,b,n=1: (b-a) * rand.random_sample((1,n)) + a

n = 200 # datapoints per category

# circle data
radius = 5
r = np.append(uniform(0, radius*.5, n//2), uniform(radius*.7, radius, n//2))
angle = uniform(0, 2*np.pi, n)
xy = np.vstack((r * np.sin(angle), r * np.cos(angle)))
t = np.less(la.norm(xy, axis=0), radius*.5)
circle = (xy.T, t)

# xor data
padding = .3
x = uniform(-5, 5, n); x[x>padding] += padding; x[x<padding] -= padding
y = uniform(-5, 5, n); y[y>padding] += padding; y[y<padding] -= padding
t = np.less(x*y, 0).flatten()
xor = (np.vstack((x, y)).T, t)

# gauss data
gauss = (np.vstack((rand.normal(2, 1, (n//2, 2)), rand.normal(-2, 1, (n//2, 2)))),
         np.append(np.ones(n//2), np.zeros(n//2)))

# spiral data
def genSpiral(deltaT, n, noise = 0):
    points = np.arange(n)
    r = points / n * 5
    t = 1.75 * points / n * 2 * np.pi + deltaT
    return np.vstack((r * np.sin(t), r * np.cos(t))).T
spiral = (np.vstack((genSpiral(0, n//2), genSpiral(np.pi, n//2))),
          np.append(np.ones(n//2), np.zeros(n//2)))

# Visualize
fig = plt.figure('Available Datasets')
ax1 = plt.subplot(221); ax1.set_title('circle')
ax1.scatter(*zip(*circle[0]), c=circle[1], cmap='bwr')
ax2 = plt.subplot(222); ax2.set_title('xor')
ax2.scatter(*zip(*xor[0]), c=xor[1], cmap='bwr')
ax3 = plt.subplot(223); ax3.set_title('gauss')
ax3.scatter(*zip(*gauss[0]), c=gauss[1], cmap='bwr')
ax4 = plt.subplot(224); ax4.set_title('spiral')
ax4.scatter(*zip(*spiral[0]), c=spiral[1], cmap='bwr')
fig.canvas.draw()

In [ ]:
# debug settings (not that it would help a lot...)
theano.config.exception_verbosity='high'

The first implementation here shows a simple perceptron implementation in Theano:

In [ ]:
## LINEAR REGRESSION

N = 400 # number of samples
feats = 20 # dimensionality of features
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2)) # rand data
training_steps = 10000

x = T.matrix("x")
y = T.vector("y")
w = theano.shared(rng.randn(20), name="w")
b = theano.shared(0., name="b")

print("-------- Perceptron Model --------")
print(w.get_value(), b.get_value())

p_1 = 1 / (1 + T.exp(-T.dot(x, w)-b)) # probability that target = 1
prediction = p_1 > 0.5 # the prediction threshold
xent = -y*T.log(p_1) - (1-y)*T.log(1-p_1) # cross-entropy loss func
cost = xent.mean() + 0.01 * (w**2).sum() # the cost to minimize
gw, gb = T.grad(cost, [w, b])

train = theano.function(inputs = [x, y], outputs = [prediction, xent],
                        updates = {w : w-0.1*gw, b : b-0.1*gb})

predict = theano.function(inputs = [x], outputs = prediction)


for i in range(training_steps):
    pred, err = train(D[0], D[1])
    
print("-------- Training result -------")
print(w.get_value(), b.get_value())
print("Target values for the data: ")
print(D[1])
print("Predictions on the data: ")
print(predict(D[0]))

Now let's extend it to the MLP with the data given above:

In [ ]:
## MULTILAYER-PERCEPTRON

# define the data
N = 400
feats = 2

data, labels = xor
D = (data, labels)

x = T.matrix("x")
y = T.vector("y")
w_1 = theano.shared(rnd.randn(2,10), name="w1")
b_1 = theano.shared(np.zeros((10,)), name="b1")
w_2 = theano.shared(rnd.randn(10), name="w2")
b_2 = theano.shared(0., name="b2")

print("-------- Multilayer Model --------")
print(w_1.get_value(), b_1.get_value())
print(w_2.get_value(), b_2.get_value())

# expression graph for forward propagation
out = T.nnet.sigmoid(-T.dot(T.nnet.sigmoid(-T.dot(x,w_1)-b_1), w_2)-b_2)

prediction = out > 0.5

# cross entropy as loss function
cross_ent = -y * T.log(out) - (1-y) * T.log(1-out)
# cost function
cost = cross_ent.mean() + 0.01 * (w**2).sum()
gw_1, gb_1, gw_2, gb_2 = T.grad(cost, [w_1, b_1, w_2, b_2])

eps = 0.1
steps = 10000

# compile the model
train = theano.function(inputs = [x, y], outputs = [prediction, cross_ent],
                        updates = [(w_1, w_1-0.1*gw_1), (b_1, b_1-0.1*gb_1),
                                   (w_2, w_2-0.1*gw_2), (b_2, b_2-0.1*gb_2)])

predict = theano.function([x], prediction)

for i in range(steps):
    pred, err = train(D[0], D[1])

print("-------- Training result -------")
print(w_1.get_value(), b_1.get_value())
print(w_2.get_value(), b_2.get_value())

print("Target values for the data: ")
print(D[1])
print("Predictions on the data: ")
print(predict(D[0]))